In [1]:
import numpy as np
import pandas as pd

# reading data from the states_all csv file
data = pd.read_csv("states_all.csv")

# Our first look at the data
print(data.describe())
print(data.head())

    

              YEAR        ENROLL  TOTAL_REVENUE  FEDERAL_REVENUE  \
count  1492.000000  1.229000e+03   1.280000e+03     1.280000e+03   
mean   2004.433646  9.159308e+05   9.092082e+06     7.663723e+05   
std       7.393983  1.065280e+06   1.174519e+07     1.145242e+06   
min    1992.000000  4.386600e+04   4.656500e+05     3.102000e+04   
25%    1998.000000  2.583140e+05   2.186305e+06     1.893540e+05   
50%    2004.000000  6.483130e+05   5.079546e+06     4.033765e+05   
75%    2011.000000  1.014528e+06   1.085985e+07     8.289660e+05   
max    2017.000000  6.307022e+06   8.921726e+07     9.990221e+06   

       STATE_REVENUE  LOCAL_REVENUE  TOTAL_EXPENDITURE  \
count   1.280000e+03   1.280000e+03       1.280000e+03   
mean    4.216553e+06   4.109157e+06       9.196681e+06   
std     5.543072e+06   5.482971e+06       1.197813e+07   
min     0.000000e+00   2.209300e+04       4.816650e+05   
25%     1.153097e+06   7.158345e+05       2.165404e+06   
50%     2.537074e+06   2.055780e+06    

In [2]:
# deleting around 200 rows that contain very little data and are not useful for predicting test scores
data = data[:1229]


# dropping columns that are unimportant for predicting student's test scores
data = data.drop(["TOTAL_REVENUE", "FEDERAL_REVENUE", "STATE_REVENUE", "LOCAL_REVENUE", "GRADES_PK_G"], axis=1)
data = data.drop(["GRADES_8_G", "GRADES_KG_G", "GRADES_4_G","GRADES_12_G", "GRADES_1_8_G", "GRADES_9_12_G"], axis=1)

# dropping primary_key column because it is redundant with state/year.
# Also dropping enrollment column because it is redundant with grades_all_g column and grades_all_g column contains more data
data = data.drop(["PRIMARY_KEY", "ENROLL"], axis=1)


print(data.describe())
print(data.head())


              YEAR  TOTAL_EXPENDITURE  INSTRUCTION_EXPENDITURE  \
count  1229.000000       1.229000e+03             1.229000e+03   
mean   2003.520749       9.027848e+06             4.669665e+06   
std       6.918718       1.171103e+07             6.132330e+06   
min    1992.000000       4.816650e+05             2.655490e+05   
25%    1998.000000       2.150719e+06             1.131889e+06   
50%    2004.000000       5.122049e+06             2.602503e+06   
75%    2009.000000       1.058529e+07             5.522474e+06   
max    2015.000000       7.836596e+07             4.195426e+07   

       SUPPORT_SERVICES_EXPENDITURE  OTHER_EXPENDITURE  \
count                  1.229000e+03       1.178000e+03   
mean                   2.629256e+06       4.223547e+05   
std                    3.278371e+06       5.241372e+05   
min                    1.399630e+05       1.154100e+04   
25%                    6.206390e+05       1.004732e+05   
50%                    1.508203e+06       2.690405e+05   

In [3]:
# adds a new column to the dataframe - values will be added later
data["avg_score"] = 0

# empty list used to store indexes that contain Nan
posList = []

# iterates through each row in dataframe to add values to posList
for index, row in data.iterrows():
    if (pd.isna(row[8]) or pd.isna(row[9]) or pd.isna(row[10]) or pd.isna(row[11])):
        posList.append(index)
        
# removes rows that include Nan for a score column
for i in range(len(posList)-1,-1,-1):
    data = data.drop([posList[i]],axis=0)
    
# starts index count from 0 again
data = data.reset_index(drop=True)
    
# adds a value to the avg_score column 
for i in range(len(data.index)):
    scoreSum = data.iloc[i,8] + data.iloc[i,9] + data.iloc[i,10] + data.iloc[i,11]
    new_score = scoreSum / 4
    data.iloc[i,12] = new_score
    

 
data = data.drop(["AVG_MATH_4_SCORE", "AVG_READING_4_SCORE", "AVG_MATH_8_SCORE","AVG_READING_8_SCORE"], axis=1)

    
print(data.describe())
print(data.head())

        

              YEAR  TOTAL_EXPENDITURE  INSTRUCTION_EXPENDITURE  \
count   423.000000       4.230000e+02             4.230000e+02   
mean   2007.257683       1.046729e+07             5.409149e+06   
std       5.605708       1.324364e+07             6.945857e+06   
min    1992.000000       6.387840e+05             3.182600e+05   
25%    2003.000000       2.602185e+06             1.395808e+06   
50%    2007.000000       6.024747e+06             3.053380e+06   
75%    2011.000000       1.141156e+07             5.947704e+06   
max    2015.000000       7.836596e+07             4.195426e+07   

       SUPPORT_SERVICES_EXPENDITURE  OTHER_EXPENDITURE  \
count                  4.230000e+02       4.160000e+02   
mean                   3.059024e+06       4.794490e+05   
std                    3.697200e+06       5.856068e+05   
min                    1.949150e+05       2.191000e+04   
25%                    7.894045e+05       1.277278e+05   
50%                    1.814927e+06       3.233980e+05   

In [4]:
# adds a new column to the dataframe - values will be added later
data["cost_per_student"] = 0
data["instruction_per_student"] = 0
data["support_per_student"] = 0
data["capital_per_student"] = 0
data["other_per_student"] = 0

# empty list used to store indexes that contain Nan
posList = []

# iterates through each row in dataframe to add values to posList
for index, row in data.iterrows():
    if (pd.isna(row[2])  or pd.isna(row[3]) or pd.isna(row[4]) or pd.isna(row[6]) or pd.isna(row[7])):
        posList.append(index)
        
# removes rows that include Nan for a score column
for i in range(len(posList)-1,-1,-1):
    data = data.drop([posList[i]],axis=0)
    
# starts index count from 0 again
data = data.reset_index(drop=True)
    
# adds a value to the avg_score column 
for i in range(len(data.index)):
    cost_per_student = data.iloc[i,2] / data.iloc[i,7]
    data.iloc[i,9] = cost_per_student
    data.iloc[i,10] = data.iloc[i,3] / data.iloc[i,7]
    data.iloc[i,11] = data.iloc[i,4] / data.iloc[i,7]
    data.iloc[i,12] = data.iloc[i,6] / data.iloc[i,7]
    data.iloc[i,13] = (data.iloc[i,2]-data.iloc[i,3]-data.iloc[i,4]-data.iloc[i,6])/data.iloc[i,7]
    

    

    
print(data.describe())
print(data.head())

              YEAR  TOTAL_EXPENDITURE  INSTRUCTION_EXPENDITURE  \
count   420.000000       4.200000e+02             4.200000e+02   
mean   2007.247619       1.033717e+07             5.345493e+06   
std       5.573173       1.285973e+07             6.769995e+06   
min    1992.000000       6.387840e+05             3.182600e+05   
25%    2003.000000       2.605188e+06             1.395902e+06   
50%    2007.000000       5.983273e+06             3.052568e+06   
75%    2011.000000       1.139698e+07             5.929521e+06   
max    2015.000000       7.476609e+07             4.195426e+07   

       SUPPORT_SERVICES_EXPENDITURE  OTHER_EXPENDITURE  \
count                  4.200000e+02       4.130000e+02   
mean                   3.019030e+06       4.731830e+05   
std                    3.570319e+06       5.645585e+05   
min                    1.949150e+05       2.310700e+04   
25%                    7.909648e+05       1.279680e+05   
50%                    1.797826e+06       3.239130e+05   

In [5]:
# time to get rid of categorical data

# empty dictionary to store the offset value for each state
state_offset = {}

# sorts dataframe by "state" column
data = data.sort_values(by=['STATE'])

# finds the average score for each state
for name, group in data.groupby(['STATE']):
    total_score = 0
    total_in_state = 0
    for row_index, row in group.iterrows():
        total_score += row[8]
        total_in_state += 1
    
    state_offset[name] = total_score / total_in_state
    
# replaces the state with the offset
for i in range(len(data.index)):
    data.iloc[i,0] = state_offset[data.iloc[i,0]] - data['avg_score'].mean()
    

# empty dictionary to store the offset value for each year
year_offset = {}

# sorts dataframe by "year" column
data = data.sort_values(by=['YEAR'])

# finds the average score for each year
for name, group in data.groupby(['YEAR']):
    total_score = 0
    total_in_year = 0
    for row_index, row in group.iterrows():
        total_score += row[8]
        total_in_year += 1
    
    year_offset[name] = total_score / total_in_year
    
# replaces the year with the offset
for i in range(len(data.index)):
    data.iloc[i,1] = year_offset[data.iloc[i,1]] - data['avg_score'].mean()
    
print(data.describe())
print(data.head())

               YEAR  TOTAL_EXPENDITURE  INSTRUCTION_EXPENDITURE  \
count  4.200000e+02       4.200000e+02             4.200000e+02   
mean  -4.851992e-14       1.033717e+07             5.345493e+06   
std    3.694520e+00       1.285973e+07             6.769995e+06   
min   -1.343053e+01       6.387840e+05             3.182600e+05   
25%   -1.911131e+00       2.605188e+06             1.395902e+06   
50%    1.024526e+00       5.983273e+06             3.052568e+06   
75%    2.713903e+00       1.139698e+07             5.929521e+06   
max    3.839920e+00       7.476609e+07             4.195426e+07   

       SUPPORT_SERVICES_EXPENDITURE  OTHER_EXPENDITURE  \
count                  4.200000e+02       4.130000e+02   
mean                   3.019030e+06       4.731830e+05   
std                    3.570319e+06       5.645585e+05   
min                    1.949150e+05       2.310700e+04   
25%                    7.909648e+05       1.279680e+05   
50%                    1.797826e+06       3.2391

In [6]:
# dropping columns that aren't necesary at this point
data = data.drop(["INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"], axis=1)

# the following code is used to randomly order the data in the dataframe
import random
data["random_index"] = 0
for i in range(len(data.index)):
    data.iloc[i,10] = random.randint(0,1000)
data = data.sort_values(by=['random_index'])
data = data.drop(["random_index"], axis=1)



In [7]:
# divides data into two sets: a training set and a test set
train_data = data[0:315]
test_data = data[315:420]


train_offset_data = data[["STATE","YEAR"]]
test_offset_data = data[["STATE","YEAR"]]

X_train = train_data.drop("avg_score", axis=1)
Y_train = train_data["avg_score"]

print(train_data.describe())
print(train_data.head())
print(test_data.describe())
print(test_data.head())






             YEAR  TOTAL_EXPENDITURE  GRADES_ALL_G   avg_score  \
count  315.000000       3.150000e+02  3.150000e+02  315.000000   
mean     0.064032       1.009404e+07  8.766965e+05  249.541420   
std      3.653458       1.163583e+07  9.797019e+05    6.825009   
min    -13.430529       7.032450e+05  5.940000e+04  216.632909   
25%     -1.911131       2.728052e+06  2.828600e+05  245.927892   
50%      1.471564       6.184819e+06  6.301340e+05  250.517045   
75%      2.907198       1.138048e+07  9.680850e+05  254.110545   
max      3.839920       7.476609e+07  5.926631e+06  264.379179   

       cost_per_student  instruction_per_student  support_per_student  \
count        315.000000               315.000000           315.000000   
mean          11.739655                 6.057851             3.565345   
std            3.756903                 2.060452             1.258861   
min            5.199958                 2.670385             1.361191   
25%            9.284909                 

In [8]:
#train_data.drop(["STATE","YEAR"], axis=1)
#test_data.drop(["STATE","YEAR"], axis=1)
# with the above two lines the total squared error = 606.74 and total error = 201.09
# without: total squared error = 504.16 and total error = 172.64
# Thus we conclude that dropping the offset values completely is unnecessary here. We are best off including them
# in the linear regression algorithm.


from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
linreg.fit(X_train, Y_train)
X_test = test_data.drop("avg_score", axis=1)
Y_test = test_data["avg_score"]
Y_pred = linreg.predict(X_test)
print(Y_pred)

# Not sure what this value actually means
acc_log = round(linreg.score(X_train, Y_train) * 100, 2)
print(acc_log)


# I would still like to add a way to find the correlation of each value. The graphs should show a visual representation
# of this though.

total_sum_squared = 0
total_sum = 0
total_sum_with_state_offset = 0
total_sum_with_year_offset = 0
total_sum_with_offsets = 0

Y_test = Y_test.reset_index(drop=True)

for i in range(len(Y_test.index)):
    total_sum += abs(Y_test.iloc[i] - Y_pred[i])
    total_sum_squared += (Y_test.iloc[i] - Y_pred[i])**2
    total_sum_with_state_offset += (Y_test.iloc[i] - Y_pred[i] + test_offset_data.iloc[i,0])**2
    total_sum_with_year_offset += (Y_test.iloc[i] - Y_pred[i] + test_offset_data.iloc[i,1])**2
    total_sum_with_offsets += (Y_test.iloc[i]-Y_pred[i] + test_offset_data.iloc[i,1] + test_offset_data.iloc[i,0])**2

print(total_sum)
print(total_sum_squared)
print(total_sum_with_state_offset)
print(total_sum_with_year_offset)
print(total_sum_with_offsets)

[250.89492933 242.58558507 252.22119032 252.60027512 255.59730931
 254.95845294 249.25207153 256.25816002 254.65509881 257.02376302
 249.56882665 252.86244187 254.05737352 255.44749014 249.05476912
 255.38910208 255.21379616 248.96726087 252.29930848 247.30181474
 251.99275711 254.28907562 256.12924466 252.42448292 246.46342552
 248.23824385 245.69987925 259.2205535  240.87713397 241.14321137
 253.62880173 239.98503546 237.42460619 255.95544786 244.69827132
 232.80830172 256.26089181 255.8896197  253.23152329 236.91592699
 258.1402075  251.62805869 256.91562453 249.46962034 251.59258096
 248.79863814 227.47687672 255.37547073 254.38364131 251.84793283
 260.21552008 257.99655195 260.91268662 253.01597802 257.0532665
 253.85244186 248.97329698 233.5257679  246.5407918  259.56650649
 252.75150046 257.62369905 227.29725576 253.38019766 239.81580336
 244.24444735 249.81080066 246.37452957 244.65282745 251.09352522
 254.03401829 249.72175376 255.50564153 257.86778124 243.1416032
 246.7373639